# Creazione di un dropdown e di un multiselect

In [1]:
# importa le librerie
import pandas as pd

col1 =['A1', 'A1', 'A2','A2','A1','A1' ]     
col2= [ 'B', 'C' , 'D' ,'E' ,'F' ,'G' ]     
tabella={'col1':col1,'col2':col2}

df=pd.DataFrame(tabella)


In [2]:
from ipywidgets import widgets,interact

# Colonna 1 : dropdown , selezione singola
selcol1=widgets.Dropdown(
    options=['---']+list(df['col1'].unique()),
    #rows=10,
    description='Colonna 1',
    disabled=False
)

# Colonna  : Scelta multipla
selcol2 = widgets.SelectMultiple(
    options=['---'] ,
    description='Colonna 2:'
)

# Aggiorna selezione colonna 2 in base a scelta di colonna 1
def update_col2(*args):
    # Filtro in base a selezione 1
    df2=df[df['col1']==selcol1.value]
    # Valori unici di colonna 2
    selcol2.options=df2['col2'].unique()
    
# Osservo le variazioni di selcol1 : quando cambia valore eseguo update_col2 
selcol1.observe(update_col2,'value')

def myselections( _selcol1,_selcol2):
    
    df3 = df.copy()
    df3=df3[df3['col2'].isin(_selcol2)]
    if len(df3) > 0:
        display(df3)
    else:
        print("No data to show for current selection")
    
    


Modificando la selezione del primo dropdown, vengono aggiornati gli elementi del secondo widget( un multiselect)

In [3]:
interact(myselections, _selcol1=selcol1,_selcol2=selcol2)

interactive(children=(Dropdown(description='Colonna 1', options=('---', 'A1', 'A2'), value='---'), SelectMulti…

<function __main__.myselections(_selcol1, _selcol2)>

# Caricamendo in un dropdown dell'elenco dei file di una dir

In [4]:
import glob
mypath="/home/antoiovi/Documenti/Dati"
files=glob.glob(mypath+"*.csv")
files.sort()

In [5]:
from ipywidgets import widgets, interactive,interact,Layout

fileSel=widgets.Text()
# FILENAME
dropfile=widgets.Dropdown(
    
    options=files,
    description='Seleziona file',
    disabled=False,
    layout=Layout(width='auto')
)
def selectfile(dropfile):
    s=dropfile
    fileSel.value=str(dropfile)
    
selectfile(dropfile)
interact(selectfile,dropfile=dropfile)
    

interactive(children=(Dropdown(description='Seleziona file', layout=Layout(width='auto'), options=('/home/anto…

<function __main__.selectfile(dropfile)>

 # Date PickUp
 Creazione di due DatePicker 
 
 * DAL : data iniziale
 * Al  : data finale
 
 - Se la data iniziale è inferiore alla data iniziale , viene impostata uguale alla data del componente che è cambiato
 - Se la data finale è inferiore alla data iniziale , viene impostata uguale alla data del componente che è cambiato
 
 ### Creazione del DataFrame di esempio

In [164]:
import pandas as pd
import numpy as np
dates    =   ['20/01/2019',  '21/01/2019','22/01/2019' ]
val_col1=['A','B']
data=[]
col1=[]
col2=[]
for d in dates:
    for v in val_col1:
        data.append(d)
        col1.append(v)
        col2.append(np.random.randint(10,20))
tab={'data':data,'col1':col1,'col2':col2}
df=pd.DataFrame(tab)
df['data']=pd.to_datetime(df['data'],format='%d/%m/%Y')
#df['data']=df['data'].dt.strftime('%d/%m/%Y')
df=df.set_index('data')
display(df)
#Per accedere  al DataFrame da dentro la routine chiamata da intercat(..)
def get_DataFrame():
    return df

,col1,col2
data,,
2019-01-20,A,12
2019-01-20,B,15
2019-01-21,A,17
2019-01-21,B,12
2019-01-22,A,11
2019-01-22,B,19


### Creazione DatePicker e routines

In [215]:
%matplotlib inline
import matplotlib as plt
from ipywidgets import widgets,interact
'''
Per trasformare in datetime i valori :
    dal=datetime(start.value.year,start.value.month,start.value.day)
    al=datetime(end.value.year,end.value.month,end.value.day)
'''
from datetime import date

# DATA INIZIALE : START
start =widgets.DatePicker(
    description='Start',
    value=pd.Timestamp(df.index.min()),
    disabled=False
)
# DATA FINALE : END
end =widgets.DatePicker(
    description='End',
    value=pd.Timestamp(df.index.max()),
    disabled=False
)

#end.format="%d/%m/%Y"
#start.format="%d/%m/%Y"

#START CHANGE
def on_change_start(change):
    if change['type'] == 'change' and change['name'] == 'value':
        _end=end.value
        if _end != None:
            new_star=pd.Timestamp(change['new'])
            endd=pd.Timestamp(_end)
            if new_star>endd:
                end.value=change['new']

# END_CHANGED
def on_change_end(change):
    if change['type'] == 'change' and change['name'] == 'value':
        _start=start.value
        if _start != None:
            new_end=pd.Timestamp(change['new'])
            startd=pd.Timestamp(_start)
            if new_end<startd:
                start.value=change['new']
            

end.observe(on_change_end)
start.observe(on_change_start)

def _printer( _start,_end):
    df3=get_DataFrame()
    df3 = df.copy()
    print("Data inferiore   : {}  Data maggiore : {}".format(df3.index[0], df3.index[-1]))
    print("Selezionato  DAL : {}             AL : {}".format(_start,_end))
    print("Datepicker END :\n\t Tipo dato: {} Valore :{}".format(type(_end),_end))
    dal=datetime(start.value.year,start.value.month,start.value.day)
    al=datetime(end.value.year,end.value.month,end.value.day)
    '''
    VERIFICARE CHE IL CAMPO (IN QUESTO CASO L'INDICE) SIA IN FORMATO DATETIME
    (SE LA DATA 'E IN FORMATO STRINGA TRASFORMARE IL VALORE DEL DATEPICKER IN FORMATO
      STRINGA)
    '''
    df3 = df3[df3.index <=al]
    df3 = df3[df3.index >=dal]
    print("Valori filtrati :")
    print(df3)
    if len(df3)>0 :
        df3['col2'].plot()
    


In [216]:
interact(_printer,_start=start, _end=end)

interactive(children=(DatePicker(value=Timestamp('2019-01-20 00:00:00'), description='Start'), DatePicker(valu…

<function __main__._printer(_start, _end)>